In [4]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge bs4 --yes

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim
import geocoder
import requests 
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    bs4-4.9.1                  |                0           4 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         223 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  bs4                conda-forge/noarch::bs4-4.9.1-0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0



beautifulsoup4-4.9.1 | 163 KB    | ##################################### | 100% 
bs4-4.9.1 

In [5]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Populated_places_in_Riyadh_Province").text
soup = BeautifulSoup(data, 'html.parser')

In [6]:
neighborhoodList = []
for i in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(i.text)

In [7]:
rdf = pd.DataFrame({"Neighborhood": neighborhoodList})
rdf.shape
rdf.head()

,Neighborhood
0,Afif
1,Al Artawiyah
2,Al Bir
3,Al Zulfi
4,Al-Ghat


In [36]:
#function to get coordinates of neighborhoods

def get_latlng(neighborhood):
    lat_long_coords = None
    while(lat_long_coords is None):
        g = geocoder.arcgis('{}, Riyadh, Saudi Arabia'.format(neighborhood))
        lat_long_coords = g.latlng
    return lat_long_coords

In [37]:
coords = [get_latlng(neighborhood) for neighborhood in rdf["Neighborhood"].tolist()]
coords

[[23.901910000000044, 42.91212000000007],
 [26.51129000000003, 45.352190000000064],
 [25.288460000000043, 45.95580000000007],
 [26.29971000000006, 44.81495000000007],
 [26.02648000000005, 44.944060000000036],
 [24.7941585, 46.6204905],
 [24.473700000000065, 46.281630000000064],
 [24.07716000000005, 45.28493000000003],
 [23.68363000000005, 43.11688000000004],
 [23.83555000000007, 44.691070000000025],
 [24.636195609823698, 46.71755157958754],
 [25.922273586619884, 45.3608747201021],
 [24.595660000000066, 46.14097000000004],
 [25.598625089112794, 45.63111087590607],
 [24.42590000000007, 46.83411000000007],
 [24.616785231149162, 46.7006387829625],
 [25.59653550000001, 45.614096999999994],
 [23.49273000000005, 46.881260000000054],
 [25.115730000000042, 46.11451000000005],
 [24.676708500000004, 46.809510749999994],
 [24.16721000000007, 47.30424000000005],
 [22.29206000000005, 46.72376000000003],
 [25.91390000000007, 45.36194000000006],
 [24.607570000000067, 46.72921000000008],
 [24.682110000

In [38]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
rdf['Latitude'] = df_coords['Latitude']
rdf['Longitude'] = df_coords['Longitude']

rdf.to_csv("rdf.csv", index=False)

print(rdf.shape)
rdf.head()

(34, 3)


,Neighborhood,Latitude,Longitude
0,Afif,23.90191,42.91212
1,Al Artawiyah,26.51129,45.35219
2,Al Bir,25.28846,45.95580
3,Al Zulfi,26.29971,44.81495
4,Al-Ghat,26.02648,44.94406


In [39]:
address = 'Riyadh, Saudi Arabia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinates of Riyadh are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Riyadh are 24.6319692, 46.7150648.


In [40]:
mapr = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(rdf['Latitude'], rdf['Longitude'], rdf['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(mapr)  

mapr.save('mapr.html')
mapr

In [41]:
CLIENT_ID = 'ARH4A4350L5BU3GXRY0N3E4B0W2W54AL4OXPHAZ0RWCWQSGL' #Foursquare ID
CLIENT_SECRET = '54A1JJIEZQDIZ5Z3DTIT3SIYHN5X1DOZ1KFU2RD0BB3SNRPA' #Foursquare Secret
VERSION = '20180605' #Foursquare API version

print("ID: "+ CLIENT_ID +"\n" +"Secret: " + CLIENT_SECRET)

ID: ARH4A4350L5BU3GXRY0N3E4B0W2W54AL4OXPHAZ0RWCWQSGL
Secret: 54A1JJIEZQDIZ5Z3DTIT3SIYHN5X1DOZ1KFU2RD0BB3SNRPA


In [42]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(rdf['Latitude'], rdf['Longitude'], rdf['Neighborhood']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [43]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1358, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Afif,23.90191,42.91212,McDonald's (ماكدونالدز),23.912289,42.922177,Fast Food Restaurant
1,Afif,23.90191,42.91212,ZamZam Hotel,23.887099,42.913967,Hotel
2,Afif,23.90191,42.91212,STC الاتصالات السعودية,23.904427,42.918887,Mobile Phone Shop
3,Afif,23.90191,42.91212,Kudu (كودو),23.905506,42.913965,Sandwich Place
4,Afif,23.90191,42.91212,souq afif riyadh,23.907381,42.915165,Convenience Store


In [44]:
#Changing venue categories into numerical values for analysis using one-hot encoding

roh = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
roh['Neighborhoods'] = venues_df['Neighborhood'] 
fixed_columns = [roh.columns[-1]] + list(roh.columns[:-1])
roh = roh[fixed_columns]
print(roh.shape)
roh.head()

(1358, 179)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bath House,Big Box Store,Bike Trail,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Buffet,Burger Joint,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Castle,Chocolate Shop,Clothing Store,Coffee Roaster,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hunting Supply,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Knitting Store,Lingerie Store,Lounge,Luggage Store,Market,Massage Studio,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Noodle House,Optical Shop,Organic Grocery,Palace,Park,Pastry Shop,Perfume Shop,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Print Shop,Rental Car Location,Resort,Rest Area,Restaurant,Roof Deck,Salad Place,Sandwich Place,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Spa,Sporting Goods Shop,State / Provincial Park,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Turkish Restaurant,Vacation Rental,Video Game Store,Wings Joint,Yoga Studio
0,Afif,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Afif,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Afif,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Afif,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Afif,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0

In [45]:
rgrp = roh.groupby(["Neighborhoods"]).mean().reset_index()
print(rgrp.shape)
rgrp.head()

(31, 179)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bath House,Big Box Store,Bike Trail,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Buffet,Burger Joint,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Castle,Chocolate Shop,Clothing Store,Coffee Roaster,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hunting Supply,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Knitting Store,Lingerie Store,Lounge,Luggage Store,Market,Massage Studio,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Noodle House,Optical Shop,Organic Grocery,Palace,Park,Pastry Shop,Perfume Shop,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Print Shop,Rental Car Location,Resort,Rest Area,Restaurant,Roof Deck,Salad Place,Sandwich Place,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Spa,Sporting Goods Shop,State / Provincial Park,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Turkish Restaurant,Vacation Rental,Video Game Store,Wings Joint,Yoga Studio
0,'Uyayna,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013889,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.083333,0.0,0.0,0.12500,0.0,0.0,0.013889,0.0,0.013889,0.0,0.0,0.138889,0.0,0.000000,0.013889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.0,0.069444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013889,0.0,0.0,0.0,0.0,0.0,0.000000,0.013889,0.0,0.013889,0.0,0.013889,0.0,0.0,0.0,0.0,0.0,0.013889,0.0,0.000000,0.0,0.0,0.041667,0.0,0.013889,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027778,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0,0.041667,0.013889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013889,0.0,0.0,0.0,0.013889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013889,0.0,0.0,0.013889,0.013889,0.0,0.0,0.0,0.0
1,Afif,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.111111,0.0,0.111111,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.111111,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.111111,0.0,0.0,0.111111,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.111111,0.0,0.0,0.0,0.0,0.0

In [46]:
rmall = rgrp[["Neighborhoods","Shopping Mall"]]
rmall.head()

,Neighborhoods,Shopping Mall
0,'Uyayna,0.013889
1,Afif,0.000000
2,Al Artawiyah,0.000000
3,Al Bir,0.000000
4,Al Majma'ah,0.015873


In [47]:
K = 3

rclstr = rmall.drop(["Neighborhoods"], axis = 1)
kmeans = KMeans(n_clusters = K, random_state = 0).fit(rclstr)

In [48]:
rmrg = rmall.copy()
rmrg["Cluster Labels"] = kmeans.labels_
rmrg.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
rmrg.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,'Uyayna,0.013889,0
1,Afif,0.000000,0
2,Al Artawiyah,0.000000,0
3,Al Bir,0.000000,0
4,Al Majma'ah,0.015873,0


In [49]:
rmrg = rmrg.join(rdf.set_index("Neighborhood"), on="Neighborhood")
print(rmrg.shape)
rmrg.head()

(31, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,'Uyayna,0.013889,0,25.913641,45.35317
1,Afif,0.000000,0,23.901910,42.91212
2,Al Artawiyah,0.000000,0,26.511290,45.35219
3,Al Bir,0.000000,0,25.288460,45.95580
4,Al Majma'ah,0.015873,0,25.913900,45.36194


In [50]:
#Sorting by clusters
rmrg.sort_values(["Cluster Labels"], inplace=True)
rmrg

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,'Uyayna,0.013889,0,25.913641,45.353170
28,Tharmada'a,0.000000,0,25.118210,45.489700
27,Thadig,0.000000,0,24.732220,46.744713
26,Sudair Industrial City,0.000000,0,25.658770,45.823280
25,Shaqraa,0.016667,0,25.916569,45.360887
24,Riyadh,0.010000,0,24.682110,46.687220
23,Manfuha,0.027397,0,24.607570,46.729210
21,Jalajil,0.000000,0,24.676709,46.809511
20,H̨awţah Sudayr,0.000000,0,25.596536,45.614097
19,Huraymila,0.000000,0,25.115730,46.114510


In [51]:
mapfinal = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(K)
ys = [i+x+(i*x)**2 for i in range(K)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(rmrg['Latitude'], rmrg['Longitude'], rmrg['Neighborhood'], rmrg['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapfinal)
       
mapfinal

In [52]:
#cluster 0
rmrg.loc[rmrg['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,'Uyayna,0.013889,0,25.913641,45.353170
28,Tharmada'a,0.000000,0,25.118210,45.489700
27,Thadig,0.000000,0,24.732220,46.744713
26,Sudair Industrial City,0.000000,0,25.658770,45.823280
25,Shaqraa,0.016667,0,25.916569,45.360887
24,Riyadh,0.010000,0,24.682110,46.687220
23,Manfuha,0.027397,0,24.607570,46.729210
21,Jalajil,0.000000,0,24.676709,46.809511
20,H̨awţah Sudayr,0.000000,0,25.596536,45.614097
19,Huraymila,0.000000,0,25.115730,46.114510


In [53]:
#cluster 1
rmrg.loc[rmrg['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
22,Layla (town),0.2,1,22.29206,46.72376


In [54]:
#cluster 2
rmrg.loc[rmrg['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
13,Batha (Riyadh),0.040000,2,24.636196,46.717552
9,Al-Kharj,0.042105,2,24.167210,47.304240
18,Hotat Bani Tamim,0.076923,2,23.492730,46.881260
17,Harmah,0.050000,2,24.616785,46.700639
10,Al-Muzahmiyya,0.062500,2,24.473700,46.281630


As is shown above, we first notice that there are not a lot of malls in the capital city of Riyadh. The concentration of malls lies within cluster 0, with the availability of a smaller the number of malls in cluster 2 (downtow. The city is indeed in need of shopping malls to aid its efforts in being a tourist zone.